<a href="https://colab.research.google.com/github/EthanBerlant/Jedha_Final_Project/blob/main/Automated_NLPing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import os
from functools import reduce
from datetime import datetime
from dateutil.parser import parse
import fnmatch

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.losses import MeanAbsolutePercentageError
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

!git clone 'https://github.com/EthanBerlant/Jedha_Final_Project.git'

fatal: destination path 'Jedha_Final_Project' already exists and is not an empty directory.


In [2]:
output_dim = 128

In [3]:
# "Token based text embedding trained on English Google News 200B corpus."
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1")
def EMBED(output_dim): 
  return hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", output_shape=[output_dim], input_shape=[], dtype=tf.string)

In [60]:
def cleaning(tweet):
  new_tweet = []
  for word in tweet.split():
    if word not in fnmatch.filter(tweet.split(), 'http*') and '@' not in word and '#' not in word:
      new_tweet.append(word)
  return ' '.join(new_tweet)

In [63]:
def ModelBuilder(layer_dimensions=[128,64,32], activation='relu'):
  model = Sequential()
  for dim in layer_dimensions:
    model.add(keras.layers.Dense(dim, activation=activation))
  model.add(keras.layers.Dense(3, activation='linear'))
  model.compile(loss='MeanAbsolutePercentageError', 
                optimizer='adam', 
                metrics=['MeanAbsoluteError'])
  return model

In [123]:
def targetize(key, target):
  annoying = pd.DataFrame(embed(joining_dict[key][key]).numpy())
  annoying['date'] = joining_dict[key]['date']
  return pd.merge(annoying.groupby(by='date').mean(), target, on='date')

In [ ]:
target = pd.read_feather('Jedha_Final_Project/Data/cleaned_dataset.ftr')
SPX = [col for col in target.columns if 'SPX' in col]
SPX.insert(0,'date')
target = target[SPX].copy()
target['date'] = target['date'].apply(datetime.date)

target['SPX_close'] = target['SPX_close'].shift(-1)
target['SPX_close-7'] = target['SPX_close-7'].shift(-7)
target['SPX_close-15'] = target['SPX_close-15'].shift(-15)
target = target.dropna()

target['SPX'] = [(a,b,c) for a,b,c in zip(target['SPX_close'], target['SPX_close-7'], target['SPX_close-15'])]
target = target[['date', 'SPX']]

In [124]:
# Creating dictionary of tweet dataframes, and associated models
file_list = [file.replace('.json','') for file in os.listdir('Jedha_Final_Project/Tweets') if '.json' in file]
joining_dict = {filename : pd.read_json(f'Jedha_Final_Project/Tweets/{filename}.json').drop(['Tweet Id', 'Username'], axis=1) for filename in file_list}

for key, df in zip(joining_dict.keys(), joining_dict.values()):
  joining_dict[key] = joining_dict[key].rename(columns={'Datetime':'date','Text':key})
  joining_dict[key]['date'] = joining_dict[key]['date'].apply(datetime.date)
  joining_dict[key][key] = joining_dict[key][key].apply(cleaning)
  joining_dict[key] = targetize(key, target)

models  = {key : ModelBuilder() for key in joining_dict.keys()}

# model_BBC_1.layers[-1].bias.assign([SPX_1['SPX_close'].mean()])
# model_BBC_7.layers[-1].bias.assign([SPX_7['SPX_close-7'].mean()])
# model_BBC_15.layers[-1].bias.assign([SPX_15['SPX_close-15'].mean()])

In [ ]:
target_1  = target.copy()
target_7  = target.copy()
target_15 = target.copy()

target_1['date']  = target_1['date'].shift(-1)
target_7['date']  = target_7['date'].shift(-2*7)
target_15['date'] = target_15['date'].shift(-2*15)

target_1  = target_1[['date', 'SPX_close']].dropna()
target_7  = target_7[['date', 'SPX_close-7']].dropna()
target_15 = target_15[['date', 'SPX_close-15']].dropna()

SPX_1  = tweets[['date','text']].merge(target_1, on='date', how='inner')
SPX_7  = tweets[['date','text']].merge(target_7, on='date', how='inner')
SPX_15 = tweets[['date','text']].merge(target_15, on='date', how='inner')

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([1901.2573], dtype=float32)>

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10),
             #tf.keras.callbacks.ModelCheckpoint(filepath='Jedha_Final_Project/Model Weights/model.{epoch:02d}-{val_loss:.2f}.h5',
                                                #save_best_only=True,
                                                #save_weights_only=True,)
]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(SPX_1['text'],
                                                    SPX_1['SPX_close'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_BBC_1.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
646/646 [==============================] - 6s 7ms/step - loss: 21.4029 - mean_absolute_error: 451.3493 - val_loss: 20.3356 - val_mean_absolute_error: 429.8079
Epoch 2/600
646/646 [==============================] - 3s 4ms/step - loss: 20.0029 - mean_absolute_error: 427.6404 - val_loss: 20.0683 - val_mean_absolute_error: 420.7628
Epoch 3/600
646/646 [==============================] - 5s 8ms/step - loss: 19.7268 - mean_absolute_error: 420.6153 - val_loss: 20.0014 - val_mean_absolute_error: 428.6804
Epoch 4/600
646/646 [==============================] - 4s 6ms/step - loss: 19.4877 - mean_absolute_error: 415.3411 - val_loss: 19.7483 - val_mean_absolute_error: 409.2454
Epoch 5/600
646/646 [==============================] - 3s 4ms/step - loss: 19.2254 - mean_absolute_error: 410.1075 - val_loss: 19.4868 - val_mean_absolute_error: 409.1437
Epoch 6/600
646/646 [==============================] - 4s 5ms/step - loss: 18.9244 - mean_absolute_error: 404.0961 - val_loss: 19.3366 - val_mean

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(SPX_7['text'],
                                                    SPX_7['SPX_close-7'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_BBC_7.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
645/645 [==============================] - 4s 6ms/step - loss: 21.5352 - mean_absolute_error: 451.9191 - val_loss: 20.3641 - val_mean_absolute_error: 432.8064
Epoch 2/600
645/645 [==============================] - 3s 4ms/step - loss: 20.2662 - mean_absolute_error: 430.5675 - val_loss: 20.2040 - val_mean_absolute_error: 430.8244
Epoch 3/600
645/645 [==============================] - 3s 4ms/step - loss: 20.0066 - mean_absolute_error: 423.8133 - val_loss: 20.0410 - val_mean_absolute_error: 423.3953
Epoch 4/600
645/645 [==============================] - 3s 4ms/step - loss: 19.7645 - mean_absolute_error: 418.2492 - val_loss: 19.8662 - val_mean_absolute_error: 421.0193
Epoch 5/600
645/645 [==============================] - 3s 4ms/step - loss: 19.4550 - mean_absolute_error: 412.2196 - val_loss: 19.6185 - val_mean_absolute_error: 412.8195
Epoch 6/600
645/645 [==============================] - 3s 4ms/step - loss: 19.1742 - mean_absolute_error: 406.8737 - val_loss: 19.4840 - val_mean

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(SPX_15['text'],
                                                    SPX_15['SPX_close-15'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_BBC_15.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
645/645 [==============================] - 3s 5ms/step - loss: 21.6281 - mean_absolute_error: 449.9515 - val_loss: 20.6900 - val_mean_absolute_error: 425.5289
Epoch 2/600
645/645 [==============================] - 3s 5ms/step - loss: 20.2980 - mean_absolute_error: 428.0872 - val_loss: 20.3251 - val_mean_absolute_error: 419.6345
Epoch 3/600
645/645 [==============================] - 3s 4ms/step - loss: 20.0190 - mean_absolute_error: 420.2380 - val_loss: 20.2227 - val_mean_absolute_error: 412.0955
Epoch 4/600
645/645 [==============================] - 3s 4ms/step - loss: 19.7566 - mean_absolute_error: 414.0957 - val_loss: 20.0190 - val_mean_absolute_error: 410.0708
Epoch 5/600
645/645 [==============================] - 3s 4ms/step - loss: 19.4868 - mean_absolute_error: 408.8352 - val_loss: 19.8048 - val_mean_absolute_error: 412.0103
Epoch 6/600
645/645 [==============================] - 3s 5ms/step - loss: 19.1770 - mean_absolute_error: 403.8261 - val_loss: 19.7280 - val_mean

In [ ]:
min(model_BBC_1.history.history['val_loss'])

18.353670120239258

In [ ]:
min(model_BBC_7.history.history['val_loss'])

18.522138595581055

In [ ]:
min(model_BBC_15.history.history['val_loss'])

18.811670303344727